# User-Based Steam Game Recommenders

This notebook contains three separate recommender systems based on the ResNet, LSTM, and NCF models, respectively.

In [3]:
import numpy as np
import tensorflow as tf

In [2]:
user_id = 10  # Example user ID

In [5]:
def focal_loss(alpha=0.14, gamma=2):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, 1e-7, 1 - 1e-7)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        loss = -alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1) \
               - (1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0)
        return K.mean(loss)
    return focal_loss_fixed

#### ResNet Game Recommender

In [3]:
resnet_model = tf.keras.models.load_model('../model/balanced(best)/resnet_best(epoch34).h5', custom_objects={'focal_loss_fixed': focal_loss()})  # Load the model
X_test_num = np.load('../data/test/numerical/X_test_scaled.npy')  # Feature data for all users

In [4]:
def recommend_games_resnet(user_id, X_test_num, n_recommendations=5):
    """
    Recommend games based on the ResNet model using user_id.

    Parameters:
    user_id (int): The ID of the user to base recommendations on.
    X_test_num (numpy.ndarray): Feature data for all users.
    n_recommendations (int): The number of recommendations to return.

    Returns:
    list: A list of recommended game indices.
    """
    user_features = X_test_num[user_id].reshape(1, -1)
    
    # Predict scores for all games using the ResNet model
    all_game_scores = resnet_model.predict(X_test_num)
    
    # Sort the games based on predicted scores for the input user
    sorted_game_indices = np.argsort(-all_game_scores.flatten())  # Sort in descending order

    # Return the top n recommended games
    return sorted_game_indices[:n_recommendations]

In [5]:

recommended_games_resnet = recommend_games_resnet(user_id, X_test_num)
print(f"Recommended games for user ID {user_id} based on ResNet: {recommended_games_resnet}")


105940/105940 [==============================] - 177s 2ms/step
Recommended games for user ID 10 based on ResNet: [ 362772 2646827 3282000 1235101  876357]


#### LSTM Game Recommender

In [12]:
lstm_model = tf.keras.models.load_model('../model/balanced(best)/lstm__strongActions(epoch8).h5', custom_objects={'focal_loss_fixed': focal_loss()})  # Load the model
X_test_seq = np.load('../data/test/timeSeries_StrongAction/X_test.npy')  # Sequence feature data for all users

In [13]:
def recommend_games_lstm(user_id, X_test_seq, n_recommendations=5):
    """
    Recommend games based on the LSTM model using user_id.

    Parameters:
    user_id (int): The ID of the user to base recommendations on.
    X_test_seq (numpy.ndarray): Sequence feature data for all users.
    n_recommendations (int): The number of recommendations to return.

    Returns:
    list: A list of recommended game indices.
    """
    user_features = X_test_seq[user_id].reshape(1, -1)
    
    # Predict scores for all games using the LSTM model
    all_game_scores = lstm_model.predict(X_test_seq)
    
    # Sort the games based on predicted scores for the input user
    sorted_game_indices = np.argsort(-all_game_scores.flatten())  # Sort in descending order

    # Return the top n recommended games
    return sorted_game_indices[:n_recommendations]

In [14]:
recommended_games_lstm = recommend_games_lstm(user_id, X_test_seq)
print(f"Recommended games for user ID {user_id} based on LSTM: {recommended_games_lstm}")

65091/65091 [==============================] - 575s 9ms/step
Recommended games for user ID 10 based on LSTM: [ 999146  763983 2035244 1537871 1806069]


#### NCF Game Recommender

In [ ]:
# Load the NCF model
ncf_model = tf.keras.models.load_model('../model/balanced(best)/best_ncf_model(epoch77).h5', custom_objects={'focal_loss_fixed': focal_loss()})
X_test_user_ncf = np.load('../data/test/ncf/realworld_10%/X_test_user_encoded.npy')  # User features for NCF
X_test_item_ncf = np.load('../data/test/ncf/realworld_10%/X_test_item_encoded.npy')  # Item features for NCF
X_test_features_ncf = np.load('../data/test/ncf/realworld_10%/X_test_features.npy')  # Features for NCF

In [23]:
def recommend_games_ncf(user_id, X_test_user_ncf, X_test_item_ncf, X_test_features_ncf, n_recommendations=5):
    """
    Recommend games based on the NCF model using user_id.

    Parameters:
    user_id (int): The ID of the user to base recommendations on.
    X_test_user_ncf (numpy.ndarray): User feature data for all users.
    X_test_item_ncf (numpy.ndarray): Item feature data for all games.
    X_test_features_ncf (numpy.ndarray): Additional feature data for all games.
    n_recommendations (int): The number of recommendations to return.

    Returns:
    list: A list of recommended game indices.
    """
    user_features = X_test_user_ncf[user_id].reshape(1, -1)
    n_games = X_test_item_ncf.shape[0]
    
    # Predict scores for all games using the NCF model
    all_game_scores = []
    for game_id in range(n_games):
        game_features = X_test_item_ncf[game_id].reshape(1, -1)
        additional_features = X_test_features_ncf[game_id].reshape(1, -1)
        score = ncf_model.predict([user_features, game_features, additional_features])
        all_game_scores.append(score[0][0])
    
    all_game_scores = np.array(all_game_scores)
    
    # Sort the games based on predicted scores for the input user
    sorted_game_indices = np.argsort(-all_game_scores)  # Sort in descending order

    # Return the top n recommended games
    return sorted_game_indices[:n_recommendations]


In [ ]:
recommended_games_ncf = recommend_games_ncf(user_id, X_test_user_ncf, X_test_item_ncf, X_test_features_ncf)
print(f"Recommended games for user ID {user_id} based on NCF: {recommended_games_ncf}")

## Next Steps
- Ensure that the model files and dataset are available in the working directory.
- Evaluate the recommendations to see if they meet expectations or if further fine-tuning is needed.